In [1]:
from Model_Training import *
import warnings
from Bus import Bus
from SimulationEngine import SimulationEngine

warnings.filterwarnings('ignore')

### DISCRETE SIMULATION

In [2]:
buses_model = Buses_model()

In [11]:
##Uncomment this if you dont have the time_models.pkl file
#buses_model.generate_model()

# Situación actual

### Intervalo '07:00'-'09:00'

In [3]:
# Define Interval
start_time = '07:00'
end_time = '09:30'

# Initialize Buses

#Obligatory used:
eco_07 = Bus(id='07', start_second=0, start_place='DISNEY', cost=42.6, capacity=37, start_time=start_time, end_time=end_time, r_base=4.5, fp = 0.0052, c_hour= 3, samples=80)
eco_50 = Bus(id='50', start_second=0, start_place='A2', cost=24.8, capacity=13, start_time=start_time, end_time=end_time, r_base=12.5, fp = 0.0281, c_hour= 1, samples=80)

total_buses = [eco_07, eco_50]

N_BUSES = len(total_buses)

#Models
pax_model = Pax_model()
pax_model.generate_model(n_buses=N_BUSES)

# Run Sim
sim = SimulationEngine(total_buses, pax_model, (start_time, end_time), show_process=False)
sim.setup()
sim.run()
sim.get_metrics()

Generating Passengers (Accumulating Inter-arrival Times)...
Scheduling Buses...
Running Event Loop...

      SIMULATION RESULTS SUMMARY      

>> GENERAL PASSENGER METRICS
Total Passengers Moved: 1583
Global Avg Waiting Time: 12.27 min
Global Max Waiting Time: 38.47 min

>> METRICS BY STOP (ORIGIN)
origin  Avg_Wait_Min  Max_Wait_Min
    A2         12.53         38.47
DISNEY         12.33         36.87
 GLAXO         10.64         34.29

>> BUS UTILIZATION & COST
Bus ID Avg Occupancy  Trips Made Total Cost Total gas (L)
    07         10.3%         240 $10,224.00        196.81
    50         21.3%         240  $5,952.00         67.28


### Intervalo '13:30'-'18:00'

In [4]:
# Define Interval
start_time = '13:30'
end_time = '18:00'

# Initialize Buses (Using your params)

#Obligatory used:
eco_07 = Bus(id='07', start_second=0, start_place='DISNEY', cost=42.6, capacity=37, start_time=start_time, end_time=end_time, r_base=4.5, fp = 0.0052, c_hour= 3, samples=80)
eco_50 = Bus(id='50', start_second=0, start_place='A2', cost=24.8, capacity=13, start_time=start_time, end_time=end_time, r_base=12.5, fp = 0.0281, c_hour= 1, samples=80)

#Add more buses here to simulate them
total_buses = [eco_07, eco_50]

N_BUSES = len(total_buses)

#Models
pax_model = Pax_model()
pax_model.generate_model(n_buses=N_BUSES)

# Run Sim
sim = SimulationEngine(total_buses, pax_model, (start_time, end_time))
sim.setup()
sim.run()
sim.get_metrics()

Generating Passengers (Accumulating Inter-arrival Times)...
Scheduling Buses...
Running Event Loop...

      SIMULATION RESULTS SUMMARY      

>> GENERAL PASSENGER METRICS
Total Passengers Moved: 2114
Global Avg Waiting Time: 11.43 min
Global Max Waiting Time: 36.64 min

>> METRICS BY STOP (ORIGIN)
origin  Avg_Wait_Min  Max_Wait_Min
    A2         11.51         36.64
DISNEY         11.27         35.08
 GLAXO         11.55         34.55

>> BUS UTILIZATION & COST
Bus ID Avg Occupancy  Trips Made Total Cost Total gas (L)
    07         12.4%         240 $10,224.00        185.86
    50         32.5%         240  $5,952.00         67.09


### Intervalo '06:00'-'07:00'

In [5]:
# Define Interval
start_time = '06:00'
end_time = '07:00'

# Initialize Buses (Using your params)

#Obligatory used:
eco_07 = Bus(id='07', start_second=0, start_place='DISNEY', cost=42.6, capacity=37, start_time=start_time, end_time=end_time, r_base=4.5, fp = 0.0052, c_hour= 3, samples=80)
eco_50 = Bus(id='50', start_second=0, start_place='A2', cost=24.8, capacity=13, start_time=start_time, end_time=end_time, r_base=12.5, fp = 0.0281, c_hour= 1, samples=80)

#Add more buses here to simulate them
total_buses = [eco_07, eco_50]

N_BUSES = len(total_buses)

#Models
pax_model = Pax_model()
pax_model.generate_model(n_buses=N_BUSES)

# Run Sim
sim = SimulationEngine(total_buses, pax_model, (start_time, end_time))
sim.setup()
sim.run()
sim.get_metrics()

Generating Passengers (Accumulating Inter-arrival Times)...
Scheduling Buses...
Running Event Loop...

      SIMULATION RESULTS SUMMARY      

>> GENERAL PASSENGER METRICS
Total Passengers Moved: 1037
Global Avg Waiting Time: 9.44 min
Global Max Waiting Time: 31.83 min

>> METRICS BY STOP (ORIGIN)
origin  Avg_Wait_Min  Max_Wait_Min
    A2          9.52         28.98
DISNEY          9.68         31.83
 GLAXO          7.75         25.97

>> BUS UTILIZATION & COST
Bus ID Avg Occupancy  Trips Made Total Cost Total gas (L)
    07          6.0%         240 $10,224.00        183.00
    50         16.1%         240  $5,952.00         63.42


### Intervalo '09:30'-'13:30'

In [6]:
# Define Interval
start_time = '06:00'
end_time = '07:00'

# Initialize Buses (Using your params)

#Obligatory used:
eco_07 = Bus(id='07', start_second=0, start_place='DISNEY', cost=42.6, capacity=37, start_time=start_time, end_time=end_time, r_base=4.5, fp = 0.0052, c_hour= 3, samples=80)
eco_50 = Bus(id='50', start_second=0, start_place='A2', cost=24.8, capacity=13, start_time=start_time, end_time=end_time, r_base=12.5, fp = 0.0281, c_hour= 1, samples=80)

#Add more buses here to simulate them
total_buses = [eco_07, eco_50]

N_BUSES = len(total_buses)

#Models
pax_model = Pax_model()
pax_model.generate_model(n_buses=N_BUSES)

# Run Sim
sim = SimulationEngine(total_buses, pax_model, (start_time, end_time))
sim.setup()
sim.run()
sim.get_metrics()

Generating Passengers (Accumulating Inter-arrival Times)...
Scheduling Buses...
Running Event Loop...

      SIMULATION RESULTS SUMMARY      

>> GENERAL PASSENGER METRICS
Total Passengers Moved: 993
Global Avg Waiting Time: 10.33 min
Global Max Waiting Time: 30.66 min

>> METRICS BY STOP (ORIGIN)
origin  Avg_Wait_Min  Max_Wait_Min
    A2         10.38         30.66
DISNEY         10.06         29.82
 GLAXO         11.40         28.87

>> BUS UTILIZATION & COST
Bus ID Avg Occupancy  Trips Made Total Cost Total gas (L)
    07          5.3%         240 $10,224.00        181.75
    50         16.6%         240  $5,952.00         63.61


### Intervalo '18:30'-'22:15'

In [7]:
# Define Interval
start_time = '06:00'
end_time = '07:00'

# Initialize Buses (Using your params)

#Obligatory used:
eco_07 = Bus(id='07', start_second=0, start_place='DISNEY', cost=42.6, capacity=37, start_time=start_time, end_time=end_time, r_base=4.5, fp = 0.0052, c_hour= 3, samples=80)
eco_50 = Bus(id='50', start_second=0, start_place='A2', cost=24.8, capacity=13, start_time=start_time, end_time=end_time, r_base=12.5, fp = 0.0281, c_hour= 1, samples=80)

#Add more buses here to simulate them
total_buses = [eco_07, eco_50]

N_BUSES = len(total_buses)

#Models
pax_model = Pax_model()
pax_model.generate_model(n_buses=N_BUSES)

# Run Sim
sim = SimulationEngine(total_buses, pax_model, (start_time, end_time))
sim.setup()
sim.run()
sim.get_metrics()

Generating Passengers (Accumulating Inter-arrival Times)...
Scheduling Buses...
Running Event Loop...

      SIMULATION RESULTS SUMMARY      

>> GENERAL PASSENGER METRICS
Total Passengers Moved: 1002
Global Avg Waiting Time: 10.12 min
Global Max Waiting Time: 30.82 min

>> METRICS BY STOP (ORIGIN)
origin  Avg_Wait_Min  Max_Wait_Min
    A2          9.91         29.66
DISNEY         10.50         30.82
 GLAXO          9.77         28.93

>> BUS UTILIZATION & COST
Bus ID Avg Occupancy  Trips Made Total Cost Total gas (L)
    07          7.0%         240 $10,224.00        179.80
    50         12.2%         240  $5,952.00         62.67
